In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load datasets
test_df = pd.read_csv("sign_mnist_test.csv")
train_df = pd.read_csv("sign_mnist_train.csv")

# Extract labels and images
y_train = train_df['label'].values
X_train = train_df.drop(columns=['label']).values
y_test = test_df['label'].values
X_test = test_df.drop(columns=['label']).values

# Reshape images (28x28 grayscale)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Normalize pixel values (0-1 range)
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert labels to categorical (one-hot encoding)
num_classes = np.max(y_train) + 1  # ✅ Ensure correct class count

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Split train-validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)

# Save model
model.save("sign_language_model.h5")


C:\Users\Vaibhav\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 27s 65ms/step - accuracy: 0.2076 - loss: 2.6546 - val_accuracy: 0.8372 - val_loss: 0.6129
Epoch 2/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.7339 - loss: 0.8038 - val_accuracy: 0.9526 - val_loss: 0.2166
Epoch 3/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.8498 - loss: 0.4298 - val_accuracy: 0.9832 - val_loss: 0.0993
Epoch 4/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - accuracy: 0.9093 - loss: 0.2759 - val_accuracy: 0.9964 - val_loss: 0.0422
Epoch 5/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 19s 55ms/step - accuracy: 0.9346 - loss: 0.1969 - val_accuracy: 0.9976 - val_loss: 0.0267
Epoch 6/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 16s 46ms/step - accuracy: 0.9501 - loss: 0.1502 - val_accuracy: 0.9976 - val_loss: 0.0168
Epoch 7/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 23s 67ms/step - accuracy: 0.9606 - loss: 0.1169 - val_accuracy: 0.9995 - val_loss: 0.0073
Epoch 8/20
344/344 ━━━━━━━━━━━━━━━━━━━━ 21s 62ms/step - accuracy: 0.9596 - loss: 0.1163 - 

In [6]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("sign_language_model.h5")

# Define class labels (Make sure these match your dataset)
class_labels = [chr(i) for i in range(65, 65 + 24)]  # A to X (24 classes)
print("✅ Model Loaded Successfully!")


✅ Model Loaded Successfully!


In [9]:
# Preprocess the frame for model prediction
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
img = cv2.resize(gray, (128, 128))  # Resize to match model input size
img = img.astype("float32") / 255.0  # Normalize
img = np.expand_dims(img, axis=-1)  # Add channel dimension (1 for grayscale)
img = np.expand_dims(img, axis=0)  # Add batch dimension


In [13]:
Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3))
# Preprocess the frame for model prediction
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
img = cv2.resize(gray, (128, 128))  # Resize to match model input size
img = img.astype("float32") / 255.0  # Normalize
img = np.expand_dims(img, axis=-1)  # Add channel dimension (1 for grayscale)
img = np.expand_dims(img, axis=0)  # Add batch dimension


In [15]:
import cv2

cap = cv2.VideoCapture(0)  # Try 1 instead of 0 if it doesn’t work

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    cv2.imshow("Camera Test", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()


In [19]:
import json

with open("class_labels.json", "r") as f:
    class_labels = json.load(f)

print(class_labels)  # Should print { "0": "A", "1": "B", "2": "C", ... }


{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'I': 7, 'ISL_Dataset': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 13, 'P': 14, 'Q': 15, 'R': 16, 'S': 17, 'T': 18, 'U': 19, 'V': 20, 'W': 21, 'X': 22, 'Z': 23}


In [20]:
print(model.input_shape)


(None, 28, 28, 1)


In [21]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # ❌ REMOVE this line
img = cv2.resize(frame, (128, 128))  # Directly resize RGB frame
img = img.astype("float32") / 255.0
img = np.expand_dims(img, axis=0)  # Add batch dimension


In [ ]:
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model("sign_language_model.h5")

# Load class labels from dataset
train_csv_path = "sign_mnist_train.csv"
train_df = pd.read_csv(train_csv_path)
unique_labels = sorted(train_df['label'].unique())  # Get unique labels
class_labels = {i: str(label) for i, label in enumerate(unique_labels)}  # Map indices to class labels

# Start webcam capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and resize
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(gray, (28, 28))  # Resize to match dataset
    img = img.astype("float32") / 255.0  # Normalize
    img = np.reshape(img, (1, 28, 28, 1))  # Reshape to match model input

    # Make prediction
    prediction = model.predict(img)
    class_index = np.argmax(prediction)  # Get predicted class index
    confidence = np.max(prediction)  # Get confidence score

    # Display label if confidence is high
    if confidence > 0.5:
        label = f"{class_labels[class_index]} ({confidence*100:.2f}%)"
    else:
        label = "Unknown"

    cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Sign Language Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━

In [12]:
print(f"Predicted class index: {class_index}, Confidence: {confidence}")


Predicted class index: 10, Confidence: 0.9209259152412415


In [13]:
print(class_labels)


{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '10', 10: '11', 11: '12', 12: '13', 13: '14', 14: '15', 15: '16', 16: '17', 17: '18', 18: '19', 19: '20', 20: '21', 21: '22', 22: '23', 23: '24'}


In [2]:
if confidence > 0.7:  # Increase threshold from 0.5 to 0.7
    label = f"{class_labels[class_index]} ({confidence*100:.2f}%)"
else:
    label = "Unknown"


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, zoom_range=0.2)
